# Importing Libraries

In [6]:
!pip install langchain sentence-transformers faiss-cpu transformers pypdf2 langchain-community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 18.3 MB/s eta 0:00:00


In [4]:
# === Load PDF ===
from langchain_community.document_loaders import PyPDFLoader

# === Split into chunks ===
from langchain.text_splitter import RecursiveCharacterTextSplitter

# === Embeddings ===
from langchain.embeddings import HuggingFaceEmbeddings

# === Vector database ===
from langchain.vectorstores import FAISS

# === Local LLM ===
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

# === Retrieval chain ===
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# Loading pdf


In [75]:
loader = PyPDFLoader("ML-Questions.pdf")
documents = loader.load()
print(f"✅ Loaded {len(documents)} pages from the PDF.\n")

✅ Loaded 3 pages from the PDF.



In [76]:
print("=== First page content preview ===")
print(documents[0].page_content[:500])

=== First page content preview ===
Question 1: What is supervised learning? 
Supervised learning is a type of machine learning where a model learns from labeled 
data, mapping inputs to known outputs to make predictions. 
 
Question 2: What is unsupervised learning? 
Unsupervised learning involves training on unlabeled data to discover hidden patterns 
or groupings within the data. 
 
Question 3: What is reinforcement learning? 
Reinforcement learning trains an agent to make sequences of decisions by rewarding 
or penalizing its 


# Chunk the text

In [95]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=30
)


chunks = splitter.split_documents(documents)

print(f"✅ Created {len(chunks)} text chunks.\n")


print("=== First chunk preview ===")
print(chunks[0].page_content)

✅ Created 9 text chunks.

=== First chunk preview ===
Question 1: What is supervised learning? 
Supervised learning is a type of machine learning where a model learns from labeled 
data, mapping inputs to known outputs to make predictions. 
 
Question 2: What is unsupervised learning? 
Unsupervised learning involves training on unlabeled data to discover hidden patterns 
or groupings within the data. 
 
Question 3: What is reinforcement learning? 
Reinforcement learning trains an agent to make sequences of decisions by rewarding


# Embeddings

In [96]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Load a local embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Build the FAISS index from our text chunks
db = FAISS.from_documents(chunks, embeddings)


# Setup Retriever

In [97]:
retriever = db.as_retriever(search_kwargs={"k": 3})


# Setup LLM Model

In [98]:
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.llms import HuggingFacePipeline

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

generator = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    max_new_tokens=512
)
llm = HuggingFacePipeline(pipeline=generator)


Device set to use cuda:0


# Setup Conversational Retrieval Chain


In [99]:
template = """
Use the following context to answer the question in your own words.
If the context does not clearly answer the question, just respond with "I don't know."

Context:
{context}

Question:
{question}

Answer:
"""


In [100]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# Memory to keep the chat history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")

# Create the QA chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever,
    memory=memory,
    return_source_documents=True,
    chain_type="stuff",
    combine_docs_chain_kwargs={"prompt": PromptTemplate.from_template(template)}
)

# Run the interactive chatbot

In [101]:
chat_history = []

while True:
    question = input("\nAsk a question (or type 'exit'): ")
    if question.lower() == "exit":
        break

    result = qa_chain({"question": question, "chat_history": chat_history})

    print("\nAnswer:", result["answer"])
    chat_history.append((question, result["answer"]))



Ask a question (or type 'exit'):  What is regularization?

Answer: Regularization techniques like L1 or L2 penalties discourage overly complex models, helping prevent overfitting.

Ask a question (or type 'exit'): Name common evaluation metrics for classification.

Answer: Accuracy, precision, recall, F1-score, and ROC-AUC

Ask a question (or type 'exit'): What is a neural network?

Answer: A neural network consists of interconnected layers of nodes (neurons) that learn complex patterns in data.

Ask a question (or type 'exit'): What is a recurrent neural network (RNN)?

Answer: RNNs are designed for sequential data, where outputs depend on prior inputs, useful in tasks like time series forecasting and language modeling

Ask a question (or type 'exit'): exit


# Gradio app

In [103]:
import gradio as gr
def answer_question(question, chat_history=[]):
    result = qa_chain({"question": question, "chat_history": chat_history})
    answer = result["answer"]
    return answer, chat_history

with gr.Blocks() as demo:
    gr.Markdown("# 🤖 Machine Learning PDF Q&A")
    gr.Markdown("Ask questions about the Machine Learning PDF. The app retrieves context and answers using a local LLM.")

    with gr.Row():
        question_input = gr.Textbox(label="Enter your question")
    output = gr.Markdown()

    chat_history_state = gr.State([])

    submit_btn = gr.Button("Get Answer")

    submit_btn.click(
        answer_question,
        inputs=[question_input, chat_history_state],
        outputs=[output, chat_history_state]
    )

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b4c829fbdae5e7bcb4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
